# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.build_survey_data.calibration_aliss import get_adjusted_input_data_frame


In [2]:
year = 2014
data_year = 2011
reform_key = 'aliss_tva_sociale'
scenario_kwargs = dict(year = year, data_year = 2011, reform_key = reform_key)
survey_scenario = SurveyScenario.create(**scenario_kwargs)
# 

In [3]:
adjusted_scenario_kwargs = dict(scenario_kwargs)
adjusted_scenario_kwargs.update(dict(
    data_year = None,
    input_data_frame = get_adjusted_input_data_frame(reform_key = reform_key[6:])
    ))
adjusted_survey_scenario = SurveyScenario.create(**adjusted_scenario_kwargs) 

kantar_expenditures_total:  52.035641344
bdf_expenditures_total:  52.218515456
population_kantar_total:  11968.2298584
bdf_expenditures_total: 

/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:119: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  input_data_frame.eval("age = 0 + (agepr > 30) + (agepr > 45) + (agepr > 60)",
/home/benjello/openfisca/openfisca-survey-manager/openfisca_survey_manager/statshelpers.py:122: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  ret[and_(data >= lower, data < upper)] = labels[i]
/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:508: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  'adjusted_bdf_budget_share = kan

 126.629058796


In [4]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]

In [5]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    'depenses_ht_tva_taux_super_reduit',
    'depenses_ht_tva_taux_reduit',
    'depenses_ht_tva_taux_intermediaire',
    'depenses_ht_tva_taux_plein',
    'tva_taux_super_reduit',
    'tva_taux_reduit',
    'tva_taux_intermediaire',
    'tva_taux_plein',
    'tva_total',
    ]

In [6]:
pivot_table = pd.DataFrame()
reference_pivot_table = pd.DataFrame()
adjusted_pivot_table = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
adjusted_aggregates = dict()

for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    reference_pivot_table = pd.concat([
        reference_pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    adjusted_pivot_table = pd.concat([
        adjusted_pivot_table,
        adjusted_survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = survey_scenario.compute_aggregate(variable, reference = True) / 1e9
    adjusted_aggregates[variable] = adjusted_survey_scenario.compute_aggregate(variable) / 1e9

In [7]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [8]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [9]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
         print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

depenses_ht_tva_taux_intermediaire :  183.132  -  75.594  =  107.538
depenses_ht_tva_taux_plein :  311.769  -  315.773  =  -4.004
depenses_ht_tva_taux_reduit :  24.994  -  128.528  =  -103.534
poste_agrege_01 :  128.399  -  124.14  =  4.259
tva_taux_intermediaire :  18.313  -  7.559  =  10.754
tva_taux_plein :  62.354  -  63.155  =  -0.801
tva_taux_reduit :  1.375  -  7.069  =  -5.694
tva_total :  82.183  -  77.924  =  4.259


In [10]:
for key in sorted(aggregates.keys()):
    if round(adjusted_aggregates[key] - aggregates[key], 3) != 0:
         print key, ': ', round(adjusted_aggregates[key], 3), ' - ', round(aggregates[key], 3), ' = ', round(adjusted_aggregates[key] - aggregates[key], 3)

depenses_alcools_forts :  2.923  -  2.932  =  -0.009
depenses_biere :  1.16  -  1.301  =  -0.141
depenses_ht_poste_01_10_1 :  2.868  -  2.924  =  -0.056
depenses_ht_poste_01_10_2 :  1.401  -  1.427  =  -0.025
depenses_ht_poste_01_1_1_1_1 :  12.844  -  13.376  =  -0.533
depenses_ht_poste_01_1_1_3_3 :  3.604  -  3.73  =  -0.126
depenses_ht_poste_01_1_1_4_1 :  1.418  -  1.488  =  -0.07
depenses_ht_poste_01_1_1_4_2 :  1.553  -  1.615  =  -0.063
depenses_ht_poste_01_1_1_4_3 :  0.74  -  0.851  =  -0.112
depenses_ht_poste_01_1_2_1_1 :  7.126  -  7.457  =  -0.331
depenses_ht_poste_01_1_2_3_1 :  1.03  -  1.075  =  -0.045
depenses_ht_poste_01_1_2_4_1 :  2.077  -  2.176  =  -0.099
depenses_ht_poste_01_1_2_4_2 :  8.06  -  8.374  =  -0.314
depenses_ht_poste_01_1_2_5_1 :  4.512  -  4.766  =  -0.253
depenses_ht_poste_01_1_2_6_1 :  0.47  -  0.471  =  -0.001
depenses_ht_poste_01_1_2_6_3 :  2.849  -  2.943  =  -0.094
depenses_ht_poste_01_1_3_1_1 :  2.649  -  2.841  =  -0.192
depenses_ht_poste_01_1_3_1_2

In [11]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [12]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

4.2585982890594494

In [14]:
pivot_table - reference_pivot_table

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,-4.642868,-4.644528,-5.460436,0.002166,-8.392785,-2.468447,-15.165699,-4.202855,-12.861649,-9.851310
depenses_ht_poste_01_1_1_3_3,4.908122,-4.867637,-1.016846,1.075582,0.498120,-2.518696,0.059357,-5.023906,-1.588948,-2.236611
depenses_ht_poste_01_1_1_4_1,0.646902,-3.852896,0.455866,-1.367854,1.400001,-3.134839,0.378746,-2.756363,-1.123048,-2.000386
depenses_ht_poste_01_1_1_4_2,0.428873,-1.523823,-0.217770,0.858718,-0.390349,-1.978517,-0.694690,-1.241739,-0.913529,-1.887659
depenses_ht_poste_01_1_1_4_3,-14.810141,-4.888679,-2.824973,-0.960965,-1.403536,-2.291659,-1.433907,-1.173844,-1.369920,-1.173878
depenses_ht_poste_01_1_2_1_1,-5.656052,0.777078,-3.842895,-0.469427,-0.632526,-7.126794,-3.065166,-11.164356,-10.649555,-8.286400
depenses_ht_poste_01_1_2_3_1,3.334487,-4.082676,-2.275840,1.287968,0.266768,-3.319893,-0.347503,0.986580,-0.796290,-1.214600
depenses_ht_poste_01_1_2_4_1,-5.374107,-1.082733,-1.739420,1.343312,-1.868073,-1.452209,0.818872,-1.507206,-2.325806,-2.167584
depenses_ht_poste_01_1_2_4_2,2.657109,-0.509595,-8.414228,4.354101,-4.442886,-2.831236,-10.988764,-2.730535,-6.494390,-5.987145
depenses_ht_poste_01_1_2_5_1,-6.577336,-13.131204,-4.125477,-3.603511,0.251678,-2.802884,-6.259147,-1.696926,-5.401250,-3.573537


In [15]:
adjusted_pivot_table - pivot_table

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,-4.642868,-4.644528,-5.460436,0.002166,-8.392785,-2.468447,-15.165699,-4.202855,-12.861649,-9.851310
depenses_ht_poste_01_1_1_3_3,4.908122,-4.867637,-1.016846,1.075582,0.498120,-2.518696,0.059357,-5.023906,-1.588948,-2.236611
depenses_ht_poste_01_1_1_4_1,0.646902,-3.852896,0.455866,-1.367854,1.400001,-3.134839,0.378746,-2.756363,-1.123048,-2.000386
depenses_ht_poste_01_1_1_4_2,0.428873,-1.523823,-0.217770,0.858718,-0.390349,-1.978517,-0.694690,-1.241739,-0.913529,-1.887659
depenses_ht_poste_01_1_1_4_3,-14.810141,-4.888679,-2.824973,-0.960965,-1.403536,-2.291659,-1.433907,-1.173844,-1.369920,-1.173878
depenses_ht_poste_01_1_2_1_1,-5.656052,0.777078,-3.842895,-0.469427,-0.632526,-7.126794,-3.065166,-11.164356,-10.649555,-8.286400
depenses_ht_poste_01_1_2_3_1,3.334487,-4.082676,-2.275840,1.287968,0.266768,-3.319893,-0.347503,0.986580,-0.796290,-1.214600
depenses_ht_poste_01_1_2_4_1,-5.374107,-1.082733,-1.739420,1.343312,-1.868073,-1.452209,0.818872,-1.507206,-2.325806,-2.167584
depenses_ht_poste_01_1_2_4_2,2.657109,-0.509595,-8.414228,4.354101,-4.442886,-2.831236,-10.988764,-2.730535,-6.494390,-5.987145
depenses_ht_poste_01_1_2_5_1,-6.577336,-13.131204,-4.125477,-3.603511,0.251678,-2.802884,-6.259147,-1.696926,-5.401250,-3.573537
